In [1]:
#Qauntum Pennylnane Quantum GNN (QGRNN-style) with Angle Embeddings


In [1]:
# 📌 Step 1: Imports and Data
import pennylane as qml
from pennylane import numpy as np
import torch
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch_geometric.utils import dense_to_sparse

# Load your pre-computed and normalized embeddings
raw_embeddings = np.load("compressed_embeddings.npy")  # shape: (1024, 16)
features = torch.tensor(raw_embeddings, dtype=torch.float32)
features_norm = (features - features.min()) / (features.max() - features.min()) * torch.pi  # [0, π]

# Load your graph (example: full correlation adjacency)
adj_matrix = torch.load("adj_matrix.pt")  # Should be (1024, 1024), binary
edge_index, _ = dense_to_sparse(adj_matrix)

# Optionally: visualize a sample subgraph
G = nx.from_numpy_array(adj_matrix.numpy())
nx.draw(G.subgraph(range(20)), node_size=30, with_labels=False)


In [2]:
#Step 2: Quantum Circuit and Model
# 📌 Quantum device and QNode config
n_qubits = 8
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev, interface="torch")
def qgnn_circuit(inputs, weights):
    for i in range(n_qubits):
        qml.RY(inputs[i], wires=i)
    qml.templates.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# Define a hybrid model wrapper
class QGNNLayer(torch.nn.Module):
    def __init__(self, n_qubits, n_layers):
        super().__init__()
        self.n_qubits = n_qubits
        self.n_layers = n_layers
        self.weights = torch.nn.Parameter(0.01 * torch.randn(n_layers, n_qubits))

    def forward(self, x_batch):
        outputs = []
        for x in x_batch:
            out = qgnn_circuit(x[:self.n_qubits], self.weights)
            outputs.append(torch.stack(out))
        return torch.stack(outputs)


In [3]:
#Step 3: Training Loop with Real Labels
# Dummy label setup for testing
labels = torch.randint(0, 2, (1024,))  # Replace with real labels when ready

# Subset: restrict to match qubit count
X = features_norm[:, :n_qubits]
y = labels

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# QGNN model
model = QGNNLayer(n_qubits=n_qubits, n_layers=3)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
epochs = 20
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    logits = model(X_train)
    loss = loss_fn(logits, y_train)
    loss.backward()
    optimizer.step()
    
    # Validation
    model.eval()
    with torch.no_grad():
        val_logits = model(X_test)
        val_loss = loss_fn(val_logits, y_test)
        val_pred = val_logits.argmax(dim=1)
        val_acc = (val_pred == y_test).float().mean()
    
    print(f"Epoch {epoch+1} | Train Loss: {loss.item():.4f} | Val Loss: {val_loss.item():.4f} | Val Acc: {val_acc:.4f}")


Epoch 1 | Train Loss: 2.5878 | Val Loss: 2.5924 | Val Acc: 0.0000
Epoch 2 | Train Loss: 2.5862 | Val Loss: 2.5898 | Val Acc: 0.0000
Epoch 3 | Train Loss: 2.5835 | Val Loss: 2.5860 | Val Acc: 0.0000
Epoch 4 | Train Loss: 2.5798 | Val Loss: 2.5812 | Val Acc: 0.0000
Epoch 5 | Train Loss: 2.5751 | Val Loss: 2.5753 | Val Acc: 0.0000
Epoch 6 | Train Loss: 2.5692 | Val Loss: 2.5683 | Val Acc: 0.0000
Epoch 7 | Train Loss: 2.5623 | Val Loss: 2.5603 | Val Acc: 0.0000
Epoch 8 | Train Loss: 2.5543 | Val Loss: 2.5511 | Val Acc: 0.0000
Epoch 9 | Train Loss: 2.5452 | Val Loss: 2.5408 | Val Acc: 0.0000
Epoch 10 | Train Loss: 2.5351 | Val Loss: 2.5296 | Val Acc: 0.0000
Epoch 11 | Train Loss: 2.5239 | Val Loss: 2.5173 | Val Acc: 0.0000
Epoch 12 | Train Loss: 2.5118 | Val Loss: 2.5040 | Val Acc: 0.0000
Epoch 13 | Train Loss: 2.4987 | Val Loss: 2.4899 | Val Acc: 0.0000
Epoch 14 | Train Loss: 2.4847 | Val Loss: 2.4750 | Val Acc: 0.0000
Epoch 15 | Train Loss: 2.4699 | Val Loss: 2.4594 | Val Acc: 0.0000
Epoc